## PyTorch Workflow

Let's explore a an example PyTorch end-to-end workflow.

In [2]:
what_were_covering = {1: "data (prepare and load)",
                      2: "Build the model",
                      3: "Fitting the model to data (training)",
                      4: "making predictions and evaluating a model (inference)",
                      5: "Saving and loading a model",
                      6: "Putting it all together"}
what_were_covering

{1: 'data (prepare and load)',
 2: 'Build the model',
 3: 'Fitting the model to data (training)',
 4: 'making predictions and evaluating a model (inference)',
 5: 'Saving and loading a model',
 6: 'Putting it all together'}

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

'2.1.2+cu121'

## Data (preparing and loading)

In [5]:
# Create *known* parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(1)
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [6]:
len(X), len(y)

(50, 50)

### Splitting data into training and test sets